In [1]:
import torch
from torch import nn
#wraps an iterable around a Dataset
from torch.utils.data import DataLoader
#a Dataset stores samples aand their corresponding labels
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt

In [2]:
#this tutorial uses torchvision, a CV domain-specic package

#the torchvision.datasets module contains Dataset objects for many real-world vision data such as CIFAR, and COCO. this tutorial uses FashionMNIST.

In [3]:
#download trainign data from open datsets
training_data = datasets.FashionMNIST(root="data", train=True, download=True, transform=ToTensor())

#download test data from open datasets
test_data = datasets.FashionMNIST(root="data", train=False, download=True, transform=ToTensor())

In [4]:
#dataloader supports functions such as automatic batching, smapling, shuffling, and multiprocess data loading. 

#here we defines a batch size of 64, i.e. each element in teh dataloader iterable will return a batch of 64 features and labels
batch_size = 64

#create data loader
train_dataloader = DataLoader(training_data, batch_size = batch_size)
test_dataloader = DataLoader(test_data, batch_size = batch_size)

#N is number of sambles (batch size)
#C is the channel dimension (greyscale only has 1 diemnsion)
#H is the height of the image
#W is the width of the image
for X, y in test_dataloader:
    print("shape of X[N, C, H, W]:", X.shape)
    print("shape of y: ", y.shape, y.dtype)
    #show only one i guesss
    break

shape of X[N, C, H, W]: torch.Size([64, 1, 28, 28])
shape of y:  torch.Size([64]) torch.int64


In [5]:
#get cpu or gpu device for training
#inline if?
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"using {device} device")

using cuda device


In [6]:
#define the model
# 1. layers are specified in the __init__ function
# 2. how data passes through the network is defined in the forward function
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        
        #this model defines a three layer feed-forward neural network
        #reshapes input into a one dimensional tensor
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            #applies a linear transformation to the data
            nn.Linear(28*28, 512),
            #applies the Rectified Linear Unit function to the output to add non-linearity to the model
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )
    
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
    
model = NeuralNetwork().to(device)

print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


In [7]:
#defining the optimizer

#we need a loss function and an optimizer to tune the model for FashionMNIST

loss_fn = nn.CrossEntropyLoss()
# lr, or learning rate, adjusts how big the changes to the parameters to the given model are (a bigger lr changes the parameters more)
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

#in a single training loop, this model makes predictions of the training set (fed in batches), and backpropagates the prediction error to adjust the model's parameters
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    #iterate over the dataloader
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        
        #compute prediction error
        pred = model(X)
        #after the data is ran through the model, the output of the model is compared to the label and a loss is returned
        loss = loss_fn(pred, y)
        
        #https://discuss.pytorch.org/t/what-step-backward-and-zero-grad-do/33301
        #zero_grad clears any old gradients (which may automatically be adjusted) from the last step (otherwise the gradients would accumulate)
        optimizer.zero_grad()
        #computes the derivative of loss with respect to the parameters (or anythign requiring gradients) with backpropagation
        loss.backward()
        #causes the optimizer to take a step based on teh gradients of the parameters
        optimizer.step()
        
        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

In [8]:
#we can then check the models performance by checkign it against the test dataset
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
            
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [9]:
#training over epochs
#(hopefully) the loss will decrease and the accuracy will increase every Epoch

epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
    
print("Done!")

Epoch 1
-------------------------------
loss: 2.303354 [    0/60000]
loss: 2.289200 [ 6400/60000]
loss: 2.284564 [12800/60000]
loss: 2.285887 [19200/60000]
loss: 2.254572 [25600/60000]
loss: 2.266069 [32000/60000]
loss: 2.250010 [38400/60000]
loss: 2.236146 [44800/60000]
loss: 2.262497 [51200/60000]
loss: 2.236424 [57600/60000]
Test Error: 
 Accuracy: 45.1%, Avg loss: 2.220217 

Epoch 2
-------------------------------
loss: 2.236279 [    0/60000]
loss: 2.214337 [ 6400/60000]
loss: 2.200476 [12800/60000]
loss: 2.218221 [19200/60000]
loss: 2.127239 [25600/60000]
loss: 2.174045 [32000/60000]
loss: 2.146591 [38400/60000]
loss: 2.116508 [44800/60000]
loss: 2.186558 [51200/60000]
loss: 2.142334 [57600/60000]
Test Error: 
 Accuracy: 44.7%, Avg loss: 2.101747 

Epoch 3
-------------------------------
loss: 2.138338 [    0/60000]
loss: 2.097630 [ 6400/60000]
loss: 2.070342 [12800/60000]
loss: 2.109786 [19200/60000]
loss: 1.931500 [25600/60000]
loss: 2.045770 [32000/60000]
loss: 1.997767 [38400/

In [10]:
#saving the model
torch.save(model.state_dict(), "quickstart model.pth")
print("saved Pytorch model to 'quicstart model.pth'")

saved Pytorch model to 'quicstart model.pth'


In [11]:
#loading models

#recreate the model structure
new_model = NeuralNetwork()
#then load the state dictionary into it
new_model.load_state_dict(torch.load("quickstart model.pth"))

<All keys matched successfully>

In [12]:
#now the model can be used to make predictions!
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]

x.to(device)
y.to(device)

with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

AttributeError: 'int' object has no attribute 'to'

In [15]:
print(test_data[:5])

ValueError: only one element tensors can be converted to Python scalars